In [ ]:
import csv

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger_eng', quiet=True)

from datetime import datetime, timedelta
#This is a helper function for calculating transcript times
def add_seconds(start_time, seconds_to_add):
  # Parse the start time
  time_obj = datetime.strptime(start_time, '%H:%M:%S')
  # Add the seconds
  new_time = time_obj + timedelta(seconds=seconds_to_add)
  # Format the new time back to string
  return new_time.strftime('%H:%M:%S')

#This is where the unzipped corpus file is stored in Colab file storage
CORPUS_FILE_PATH = 'DH2024_Corpus_Release/'

#We have 4 state sets
VALID_STATES=["CA", "FL", "NY", "TX"]

#Each state set has 9 csv files for each session year
CSV_FILENAMES=['bills','committeeHearings', 'committeeRosters',
               'committees', 'hearings', 'legislature',
               'people','speeches','videos']

#Load data from CSVs into a Python object to reference later
#Input:
#  Required: file_name (type:String) (Ex: speeches, bills, etc)
#  Optional: states (type:List of Strings or None) (Ex: ["CA"], ["FL,TX"])
#     -If not specified (states=None), function returns data for all states
#  Optional: years (type:List of Ints or None) (Ex:[2018], [2015,2016])
#     -If not specified (years=None), function returns data for all valid years
#Output:
#  Payload (Type: Dict) (Ex: {column_headers:['pid','cid','date'], rows:[[0,2,2018],[2,1,2018]]})
def load_content(file_name, states=None, years=None):
  #Only accept valid states, Corpus only contains data on CA, FL, NY, and TX legislations
  if states is not None and not all(item in VALID_STATES for item in states):
    raise Exception("Invalid State Abbv(s), corpus only contains data on CA, FL, NY, and TX")
  #Only accept valid file names from corpus, like speeches, bills, etc.
  if file_name not in CSV_FILENAMES:
    raise Exception("Invalid filename, must be one of the 9 files provide")
  #Only accept years belonging to a valid legislative session. (2017-2018 for all states, 2015-2016 for CA)
  if years is not None and ((not all(item > 2015 for item in years) and "CA" not in states) or (not all(item <= 2018 for item in years))):
    raise Exception("""Data for requested year not included in corpus.
     Valid session_years are 2017 and 2018 for all states provided. 2015 and 2016 are valid years for CA.""")

  payload = {}
  header_row = True

  #If no states specified, retrieve relevant files for all valid states
  if states is None:
    states = VALID_STATES

  #If no years/session specified, retrieve data for all valid state legislative session years
  if years is None:
    if "CA" in states:
      years= [2015,2016,2017,2018]
    else:
      years = [2017,2018]

  #The following code block operates as follows:
  # For every state and year requested, read the relevant CSV file(s), then
  # load it into a python object (payload) which is returned to user
  for state in states:
    FILE_PATHS = []

    #Build the filepaths to the correct data location given the states and years provided
    #Years 2017 and 2018 are valid inputs that belong to the same 2017-2018 session
    if 2017 in years or 2018 in years:
      FILE_PATHS.append(CORPUS_FILE_PATH + state + "/2017-2018/CSV/" + file_name + ".csv")

    #CA has 2 valid legislative sessions (2015-2016 and 2017-2018)
    #This means the entirety of CA data is located in more than one folder, unlike other states.
    #Looping through a list of filepaths allows us to handle this corner case
    if state == "CA" and (2015 in years or 2016 in years):
      FILE_PATHS.append(CORPUS_FILE_PATH + state + "/2015-2016/CSV/" + file_name + ".csv")

    for FILE_PATH in FILE_PATHS:
      #Open the file to read
      with open(FILE_PATH, newline='', encoding='utf-8') as csvfile:
        rows = csv.reader(csvfile, delimiter=',')
        #Read CSV row by row
        for row in rows:
          #The first row of every CSV we visit is the header row, containing the names for each column
          # We will add this to the payload only once, as every CSV we read after this will be the same headers
          if header_row:
            payload['column_headers'] = row
            #Sets up 'rows' in payload where we will store future records
            payload['rows'] = []
            header_row = False
            continue
          #Load CSV Into payload row by row
          payload['rows'].append(row)

  return payload

#Retrieve Committee Name & ID, hearing date, list of videos of the hearing, and state for a given HID
#Input:
#  Required: HID (type:Positive Int) (Ex: 10003)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   Dictionary
#   If no matches exist does not exist in database, returns []
def get_metadata_hearing(hid, hearings=None, videos=None):
  HID_IDX=0
  CID_IDX=4
  CNAME_IDX=8
  HDATE_IDX=1
  STATE_IDX=3

  if hearings is None:
    hearings=load_content("hearings")
  hid = str(hid)

  hearingData=None
  for row in hearings['rows']:
    if hid == row[HID_IDX]:
      hearingData={'hid':row[HID_IDX],'cid':row[CID_IDX],'cname':row[CNAME_IDX],'hearing_date':row[HDATE_IDX],'state':row[STATE_IDX]}
  if hearingData is None:
    return {}

  #vids, videos = videos_from_hid(hid)
  #hearingData['vids'] = vids
  hearingData['videos'] = videos_from_hid(hid)

  return hearingData


#Retrieve the video ids and video URLs associated with a hearing
#Input:
#  Required: Hearing ID (type:Positive Int) (Ex: 10003)
#  Optional: videos
#      -If searching through from a specific state or session, pass in
#         videos=load_content("videos", specific states, specific years)
#Output:
#   (Type: List of[[Int,String]]) (Ex: [[0, "video0.mp4"], [1,"video1.mp4"]])
#   If no matches exist does not exist in database, returns []
def videos_from_hid(hid, videos=None):
  HID_IDX = 2
  VID_IDX = 0
  URL_IDX = 7

  if videos is None:
    videos=load_content("videos")

  vids = []
  video_files=[]
  hid=str(hid)

  for row in videos['rows']:
    if row[HID_IDX] == hid:
      vids.append((row[VID_IDX],row[URL_IDX])) #tuple of (video id, url)
  return vids

#Create transcript for a given bill discussion with metadata
#Input:
#  Required: HID (type:Positive Int) (Ex: 10003)
#  Required: BID (type: String)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   Dictionary and Transcript
#   If no matches exist does not exist in database, returns []
def get_hearing_transcript(hid, bid, speeches=None):
  PID_IDX=1
  BID_IDX=4
  HID_IDX=3
  VID_START_IDX = 9
  VID_END_IDX = 10
  LAST_NAME_IDX = 14
  FIRST_NAME_IDX = 15
  TEXT_IDX = 16
  STARTING_TIME_IDX = 11
  if speeches is None:
    speeches=load_content("speeches")

  hid=str(hid)

  lines = []
  for row in speeches['rows']:
    if hid == row[HID_IDX] and bid == row[BID_IDX]:
      offset_time = add_seconds("00:00:00", int(row[STARTING_TIME_IDX]))
      tokenized_text = nltk.word_tokenize(row[TEXT_IDX])
      sid = SentimentIntensityAnalyzer()
      line = {'video start':row[VID_START_IDX],'video end':row[VID_END_IDX],'offset':offset_time,'bid':row[BID_IDX],
              'first name':row[FIRST_NAME_IDX],'last name':row[LAST_NAME_IDX],'pid':row[PID_IDX],'text':row[TEXT_IDX], 'tokenized_text': tokenized_text, 'pos_tags': nltk.pos_tag(tokenized_text), 'sentiment': sid.polarity_scores(row[TEXT_IDX])}
      lines.append(line)
  return lines

#Helper function to get bill discussion metadata
def bill_discussion_info(hid, bid, hearings=None, speeches=None, videos=None):
  return {"metadata":get_metadata_hearing(hid, hearings,videos), "transcript":get_hearing_transcript(hid, bid, speeches)}


# accepts the output from get_hearing_transcript() and prints a transceript.
def pprint_discussion(metadata, transcript_info):
  videos = {}
  for vid, url in metadata['videos']:
    videos[vid] = url

  print()
  print(f"] Discussion of {metadata['state']} {metadata['cname']} held on {metadata['hearing_date']}")
  print(f"] {len(videos.keys())} videos")
  print("] printing transcript: ")

  prev_video = -1
  for line in transcript_info:
    video = line['video start']
    if video != prev_video:
      print()
      print(f"] Discussing {line['bid']}")
      print(f"] Video: {videos[line['video start']] if line['video start'] in videos else line['video start']}")
      print()
      prev_video = video
    print(f"[{line['offset']}] {line['first name']} {line['last name']}: ")
    print(f"\t{line['text']}")
  print()


In [ ]:
discussion = bill_discussion_info(53120, "FL_20170HB883")

In [ ]:
#print(discussion['transcript'][0]['text'])
for item in discussion['transcript']:
  print(f"{item['text']}\n\n")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

In [ ]:
def createTaggedOutputs(discussion):
    tagged_outputs = []
    for item in discussion['transcript']:
        print(f"{item['text']}\n\n")
        ner_results = nlp(item['text'])
        tagged_outputs.append(ner_results)  
    return tagged_outputs


def merge_org_entities(ner_output):
    merged_entities = []
    current_org = None

    for entry in ner_output:
        if entry['entity'] == 'B-ORG':
            # Start a new organization
            if current_org:
                merged_entities.append(current_org)
            current_org = {
                'entity': 'ORG',
                'score': entry['score'],
                'word': entry['word'],
                'start': entry['start'],
                'end': entry['end']
            }
        elif entry['entity'] == 'I-ORG' and current_org:
            # Append to the current organization
            current_org['word'] += f" {entry['word']}"
            current_org['score'] = min(current_org['score'], entry['score'])  # Take the minimum score
            current_org['end'] = entry['end']
        else:
            # Finalize the current organization if it exists
            if current_org:
                merged_entities.append(current_org)
                current_org = None
            # Add non-ORG entities directly
            merged_entities.append(entry)
    
    # Add the last organization if it's not already added
    if current_org:
        merged_entities.append(current_org)

    return merged_entities




In [ ]:
import csv
from Levenshtein import distance as levenshtein_distance  # Install with: pip install python-Levenshtein

def validate_org_entities(output_list, csv_file_path):
    validated_texts = []

    # Read the CSV file values
    with open(csv_file_path, 'r') as file:
        csv_values = [row[1] for row in csv.reader(file)]  # Assuming single-column CSV
    
    # Iterate over model outputs
    for entry in output_list:
        for entity in merge_org_entities(entry):
            entity_type = entity['entity']
            text = entity['word']

            #Check if ORG, concatenated B- and I- ORG above
            if entity_type == "ORG":
                # Compare the text to each value in the CSV file
                print(text)
                for csv_value in csv_values:
                    if levenshtein_distance(text, csv_value) < 2:
                        validated_texts.append(text)
                        break  # Avoid duplicates if text matches multiple CSV values

    return validated_texts


csv_path = 'organizations.csv'  # Path to your CSV file

data = [['FL_20170HB1209', '53253', '2017', '2017-03-14'],
 ['FL_20170HB1203', '53202', '2017', '2017-03-21'],
 ['FL_20170SB1206', '52788', '2017', '2017-03-27'],
 ['FL_20170HB1209', '53154', '2017', '2017-03-29'],
 ['FL_20170HB1205', '53106', '2017', '2017-04-06'],
 ['FL_20170HB1203', '53104', '2017', '2017-04-13'],
 ['FL_20170HB1209', '53089', '2017', '2017-04-20'],
 ['FL_20170SB1206', '53015', '2017', '2017-04-25'],
 ['FL_20170HB1203', '52820', '2017', '2017-04-26'],
 ['FL_20170HB1201', '53021', '2017', '2017-04-27'],
 ['FL_20170HB1203', '53059', '2017', '2017-05-02'],
 ['FL_20170SB1208', '253077', '2017', '2018-01-16'],
 ['FL_20170SB1120', '253154', '2017', '2018-01-18'],
 ['FL_20170HB1201', '253220', '2017', '2018-01-24'],
 ['FL_20180SB1200', '253400', '2017', '2018-02-06'],
 ['FL_20180HB1201', '253416', '2017', '2018-02-07'],
 ['FL_20180SB1200', '253507', '2017', '2018-02-14'],
 ['FL_20180HB1201', '253520', '2017', '2018-02-15'],
 ['FL_20180HB1201', '254054', '2017', '2018-03-01'],
 ['FL_20180HB1201', '254138', '2017', '2018-03-08'],
 ['FL_20180HB1201', '254432', '2017', '2018-03-09']]

for entry in data:
  bid = entry[0]
  hid = entry[1]
  discussion = bill_discussion_info(hid, bid)
  tagged_outputs = createTaggedOutputs(discussion)
  validated_texts = validate_org_entities(tagged_outputs, "orgConcepts.csv")
  if validated_texts:
      print(tagged_outputs)
  print("Validated Texts:", validated_texts)
